In [2]:
import torch
import torch.nn as nn
torch.manual_seed(1)

In [3]:
class cnn(nn.Module):

    def __init__(self):

        super().__init__()        
        self.layer1 = nn.Conv2d(1, 2, kernel_size=3, bias=False)

    def forward(self, x):
        
        out = self.layer1(x)
        return out

In [4]:
x = torch.randn(1, 1, 5, 5)
model = cnn()
out = model(x)

In [5]:
out.shape

torch.Size([1, 2, 3, 3])

In [6]:
w = model.state_dict()['layer1.weight']
w.shape

torch.Size([2, 1, 3, 3])

In [7]:
out

tensor([[[[ 4.2621e-01, -6.7413e-03, -1.7986e-01],
          [-7.3923e-01,  1.9493e-01, -2.4097e-01],
          [ 3.3699e-04, -1.5413e-01, -1.1222e+00]],

         [[-1.9412e-01, -9.4184e-01,  9.3807e-01],
          [-4.2307e-01,  2.4988e-02,  5.0152e-01],
          [-2.7248e-01,  5.9709e-01,  7.1902e-02]]]],
       grad_fn=<ThnnConv2DBackward0>)

In [8]:
(w[0] * x[0, :, 0:3, 0:3]).sum()

tensor(0.4262)

In [9]:
# 输入与输出关系方程式
bs, n_channels, width, height = x.shape
k_size = w.shape[-1]
n_row = (width - k_size + 1)**2
print(f'n_row: {n_row}')
n_unknown = width*height
print(f'n_unknown: {n_unknown}')
A_w = torch.zeros(size=(n_row, n_unknown))
A_w.shape

n_row: 9
n_unknown: 25


torch.Size([9, 25])

In [10]:
w

tensor([[[[-0.1410,  0.2500,  0.0039],
          [-0.1756,  0.1713, -0.1769],
          [ 0.0980, -0.0963, -0.0365]]],


        [[[-0.3205, -0.1589,  0.1809],
          [-0.0810,  0.3320,  0.2672],
          [-0.0156, -0.2225,  0.2030]]]])

In [11]:
n_multi_one_row = width - k_size + 1
for k in range(n_multi_one_row):
    for i in range(n_multi_one_row):
        for j in range(k_size):
            A_w[k*n_multi_one_row + i, j*width+i+k*width: j*width+i+k*width+k_size] = w[0, 0, j, :]

In [12]:
A_w

tensor([[-0.1410,  0.2500,  0.0039,  0.0000,  0.0000, -0.1756,  0.1713, -0.1769,
          0.0000,  0.0000,  0.0980, -0.0963, -0.0365,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000, -0.1410,  0.2500,  0.0039,  0.0000,  0.0000, -0.1756,  0.1713,
         -0.1769,  0.0000,  0.0000,  0.0980, -0.0963, -0.0365,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000, -0.1410,  0.2500,  0.0039,  0.0000,  0.0000, -0.1756,
          0.1713, -0.1769,  0.0000,  0.0000,  0.0980, -0.0963, -0.0365,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.1410,  0.2500,  0.0039,
          0.0000,  0.0000, -0.1756,  0.1713, -0.1769,  0.0000,  0.0000,  0.0980,
         -0.0963, -0.0365,  0.0000,  0.0000,  0.0000

In [13]:
x_flatten = x.reshape(-1,1)
x_flatten.shape
torch.mm(A_w, x_flatten)

tensor([[ 4.2621e-01],
        [-6.7413e-03],
        [-1.7986e-01],
        [-7.3923e-01],
        [ 1.9493e-01],
        [-2.4097e-01],
        [ 3.3698e-04],
        [-1.5413e-01],
        [-1.1222e+00]])

In [17]:
out_flatten = out.reshape(-1,1)
out_flatten

tensor([[ 4.2621e-01],
        [-6.7413e-03],
        [-1.7986e-01],
        [-7.3923e-01],
        [ 1.9493e-01],
        [-2.4097e-01],
        [ 3.3699e-04],
        [-1.5413e-01],
        [-1.1222e+00],
        [-1.9412e-01],
        [-9.4184e-01],
        [ 9.3807e-01],
        [-4.2307e-01],
        [ 2.4988e-02],
        [ 5.0152e-01],
        [-2.7248e-01],
        [ 5.9709e-01],
        [ 7.1902e-02]], grad_fn=<ReshapeAliasBackward0>)

In [26]:
import numpy as np

A_w_numpy = A_w.numpy()
print(A_w_numpy.shape)
b_w_numpy = out_flatten[0: n_row, :].detach().clone().numpy()
pred_input, _, rank, _ = np.linalg.lstsq(A_w_numpy, b_w_numpy, rcond=None)

(9, 25)


In [25]:
rank

9

In [24]:
loss_fn = nn.MSELoss()

pred_input = torch.as_tensor(pred_input).view(-1,1)
loss_fn(pred_input, x_flatten)

tensor(0.5968)

In [27]:
pred_input

array([[-0.37953302],
       [ 0.90946215],
       [ 0.1467992 ],
       [-0.99178755],
       [-0.01555615],
       [ 0.52261424],
       [-0.5170077 ],
       [-0.8962344 ],
       [-0.5225682 ],
       [ 0.69526553],
       [ 1.5921052 ],
       [-0.0812735 ],
       [-0.07212831],
       [-1.6438159 ],
       [ 0.19572845],
       [-0.58078897],
       [ 1.5966632 ],
       [ 1.1323937 ],
       [-0.21359588],
       [ 1.8455153 ],
       [-0.06195998],
       [-0.70236546],
       [-0.23952259],
       [ 1.2780426 ],
       [ 0.37722492]], dtype=float32)